## Materials configuration

This example shows how to:
   - configure various materials with the `make_material` method
   - upload textures to the device in the default 32 bit/channel format as well as in 8 bit/channel for lower memory usage

![plotoptix ray_tracing_output](https://plotoptix.rnd.team/images/materials_cfg.jpg "This notebook output")

The variety of materials is configured with a single utility function in this notebook. But you can start with predefined materials and create your variants by modifying parameters, as shown in other examples.

Make some data for a simple scene first:

In [1]:
import numpy as np

rx = (-20, 20)
rz = (-20, 20)
n = 100

x = np.linspace(rx[0], rx[1], n)
z = np.linspace(rz[0], rz[1], n)

X, Z = np.meshgrid(x, z)

# positions of blocks
data = np.stack((X.flatten(), np.full(n*n, -2), Z.flatten())).T
# XZ sizes
size_u = 0.96 * (rx[1] - rx[0]) / (n - 1)
size_w = 0.96 * (rz[1] - rz[0]) / (n - 1)

Setup the raytracer using Tkinter GUI as the output target. Note, it is important to select the background mode which supports scattering in volumes (used in some of presented materials): ``AmbientAndVolume``, ``TextureFixed``, or ``TextureEnvironment``.

In [2]:
from plotoptix import TkOptiX

rt = TkOptiX()
rt.set_param(min_accumulation_step=4,        # set more accumulation frames to get rid of the noise
             max_accumulation_frames=512,
             light_shading="Hard")           # use ligth shading best for caustics

rt.set_uint("path_seg_range", 15, 40)        # more path segments to improve simulation in volumes

exposure = 1.0; gamma = 2.2
rt.set_background_mode("TextureEnvironment") # need one of modes supporting scattering in volumes
rt.set_background("data/starmap_4k.jpg", gamma=gamma, rt_format="UByte4") # 8 bit/channel, forced with rt_format value
rt.set_background(0)
rt.set_ambient(0)

rt.set_float("tonemap_exposure", exposure)
rt.set_float("tonemap_gamma", gamma)
rt.set_float("denoiser_blend", 0.2)
rt.add_postproc("Denoiser")    # apply AI denoiser, or
#rt.add_postproc("Gamma")      # use gamma correction

rt.setup_camera("cam1", cam_type="Pinhole",
                eye=[0, 15, 1.55], target=[0, 0, 1.55], up=[1, 0, 0],
                fov=45)

rt.setup_light("light1", pos=[5.2, 8, 7], color=8, radius=1.5)
rt.setup_light("light2", pos=[-6, 8, -5], color=[10, 11, 12], radius=1.3)

Only *diffuse* material is available by default. Other materials need to be configured before using. Textures have to be set before materials are created.

In [3]:
from plotoptix.utils import read_image, make_color_2d

wood = read_image(r"data/wood.jpg", normalized=True)
alpha = np.full(wood.shape[:2] + (4,), 1, dtype=np.float32)
alpha[:,:,3] = wood[:,:,1]  # green channel copied to alpha channel

wood = wood[:,:,1]  # make a grayscale image from the green channel
wood -= np.amin(wood)
wood *= 1.0 / np.amax(wood)

# 8 bit/channel, forced with the array dtype:
rt.set_texture_2d("mask", (255*alpha).astype(np.uint8))
rt.set_texture_2d("wood", (255*wood).astype(np.uint8))
rt.set_texture_2d("wood4", (255*np.power(wood, 4)).astype(np.uint8))

rainbow = read_image(r"data/rainbow.jpg", normalized=True)
rainbow_exp = np.exp(rainbow) # use exponential space to preserve colors used as attenuation lengths in the glass

# 32 bit/channel, floating point, default:
rt.set_texture_2d("rainbow", make_color_2d(rainbow, channel_order="RGBA"))
rt.set_texture_2d("rainbow_exp", make_color_2d(rainbow_exp, channel_order="RGBA"))

In [4]:
from plotoptix.materials import make_material

m_glass_1 = make_material("Transmissive", color=0.15, color_tex="rainbow_exp")
m_glass_2 = make_material("Transmissive", color=100, radiation_length=0.1, subsurface_color=[0.7, 0.85, 1])
m_glass_3 = make_material("Transmissive", color=100, radiation_length=0.8, light_emission=0.01, subsurface_color=[0.9, 1, 1])
m_glass_4 = make_material("Transmissive", color=100, refraction_index=[1.35, 1.4, 1.48])

m_diffuse_1 = make_material("Diffuse")
m_diffuse_2 = make_material("Diffuse", color_tex="rainbow")
m_diffuse_3 = make_material("TransparentDiffuse", color_tex="mask")

m_specular_1 = make_material("Reflective", specular=0.1)
m_specular_2 = make_material("Reflective", specular=0.6)
m_specular_3 = make_material("Reflective", color=[0.01, 0.3, 0.8], specular=0.9, roughness=0.05, roughness_tex="wood4")

m_metal_1 = make_material("Reflective", metalness=1.0, roughness=0.004, roughness_tex="wood4")
m_metal_2 = make_material("Reflective", specular=0.1, metalness=1.0, metalness_tex="wood")

rt.setup_material("glass_1", m_glass_1)       # colored with the texture (note the exp color space)
rt.setup_material("glass_2", m_glass_2)       # bluish scattering in the volume
rt.setup_material("glass_3", m_glass_3)       # light emission in the volume
rt.setup_material("glass_4", m_glass_4)       # clear glass with the light refration
rt.setup_material("diffuse_1", m_diffuse_1)   # simple diffuse material
rt.setup_material("diffuse_2", m_diffuse_2)   # diffuse, textured
rt.setup_material("diffuse_3", m_diffuse_3)   # diffuse, transparency in the texture alpha channel
rt.setup_material("specular_1", m_specular_1) # slightly reflective, plastic-like surface
rt.setup_material("specular_2", m_specular_2) # a more shiny plastic-like surface
rt.setup_material("specular_3", m_specular_3) # shiny syrface with a roughness texture
rt.setup_material("metal_1", m_metal_1)       # metallic with a roughness texture
rt.setup_material("metal_2", m_metal_2)       # metalness modulated with a texture

Add objects to the scene.

In [5]:
rt.set_data("blocks", pos=data,
            c=0.8 + 0.15*np.random.randint(3, size=data.shape[0]),
            u=[size_u, 0, 0], v=[0, -1, 0], w=[0, 0, size_w],
            geom="Parallelepipeds")

rt.set_data("c_clear", pos=[-3.5, 0, -5], u=[0.25, 0, 0], v=[0, 4, 0], w=[0, 0, 4], c=0.8, mat="glass_1", geom="Parallelepipeds")
rt.rotate_geometry("c_clear", [0, 0, -np.pi/3])
rt.set_data("c_diffuse", pos=[-0.5, 0, -5], u=[0.25, 0, 0], v=[0, 4, 0], w=[0, 0, 4], c=0.8, mat="glass_2", geom="Parallelepipeds")
rt.rotate_geometry("c_diffuse", [0, 0, -np.pi/3])
rt.set_data("c_light", pos=[2.5, 0, -5], u=[0.25, 0, 0], v=[0, 4, 0], w=[0, 0, 4], c=0.8, mat="glass_3", geom="Parallelepipeds")
rt.rotate_geometry("c_light", [0, 0, -np.pi/3])

rt.set_data("s_diffuse_3", pos=[-3.1, 1.5, 1], r=1.5, c=0.9, mat="diffuse_3", geom="ParticleSetTextured")
rt.set_data("s_diffuse_2", pos=[0, 1.5, 1], r=1.5, c=0.8, mat="diffuse_2", geom="ParticleSetTextured")
rt.set_data("s_diffuse_1", pos=[3.1, 1.5, 1], r=1.5, c=0.8, mat="diffuse_1")

rt.set_data("s_specular_3", pos=[-3.1, 1.5, 4.1], r=1.5, c=[0.8, 0.9, 0.99], mat="specular_3", geom="ParticleSetTextured")
rt.set_data("s_specular_2", pos=[0, 1.5, 4.1], r=1.5, c=[0.99, 0.9, 0.8], mat="specular_2")
rt.set_data("s_specular_1", pos=[3.1, 1.5, 4.1], r=1.5, c=[0.7, 0.5, 0.1], mat="specular_1")

rt.set_data("s_metal_3", pos=[-3.1, 1.5, 7.2], r=1.5, c=0.9, mat="metal_2", geom="ParticleSetTextured")
rt.set_data("s_metal_2", pos=[0, 1.5, 7.2], r=1.5, c=0.9, mat="metal_1", geom="ParticleSetTextured")
rt.set_data("s_glass_4", pos=[3.1, 1.5, 7.2], r=1.5, c=0.9, mat="glass_4")

Open the GUI.

In [6]:
rt.start()

Close GUI window, release resources.

In [7]:
rt.close()